# Part 15: Re-ranking

### 根据rag-fusion进行索引

In [ ]:
# 加载并索引数据
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
# from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    # embedding=CohereEmbeddings()
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever()

In [ ]:
# 由llm生成多个query
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
# 请求llm生成多个query，用于rag-fusion
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()


api_url = os.getenv('API_URL')
api_key = os.getenv('API_KEY')
model_name = os.getenv('MODEL')
llm = init_chat_model(
    model_provider="openai",  # 避免langchain根据模型名自动选择供应商
    model=model_name,
    # temperature=0.0,
    api_key=api_key,
    base_url=api_url,
)

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
# 将多个query的检索结果，通过rrf打分，进行排序（与rag-fusion部分逻辑完全一样）
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

question = "What is task decomposition for LLM agents?"
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

In [ ]:
# 基于fusion的结果进行回答
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

### 使用CohereRerank进行重排序
介绍链接：https://cohere.com/blog/rerank  
langchain介绍文档：https://docs.langchain.com/oss/python/integrations/retrievers/cohere-reranker#doing-reranking-with-coherererank  
**核心思想**：将检索到的doc，通过“排序模型”计算查询和doc的相似度进行排序（类似召回和排序的逻辑），保留topk的结果，提升问答效果。  
**优势**：排序模型可以做得更加复杂，相关性计算更加准确。

In [ ]:
from langchain_community.llms import Cohere
from langchain.retrievers import  ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [ ]:
from langchain.retrievers.document_compressors import CohereRerank

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Re-rank
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(question)

# Part 16 - Retrieval (CRAG)
**核心思路**：通过构建工作流，让llm对检索结果进行反思，判断当前的检索结果是否满足需求，当质量或信息不满足回答问题的需要时，会使用web search来获取补充信息。  
参考代码：https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb

# Part 17 - Retrieval (Self-RAG)
**核心思路**：通过构建工作流，让llm对检索结果进行反思，判断当前的检索结果是否满足需求，当质量或信息不满足回答问题的需要时，改写query再次进行检索。  
参考代码：https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_self_rag.ipynb

# Part 18 - Impact of long context
视频资料：https://www.youtube.com/watch?v=SsHUNfhF32s  
PPT资料：https://docs.google.com/presentation/d/1mJUiPBdtf58NfuSEQ7pVSEQ2Oqmek7F1i4gBwR6JDss/edit?slide=id.g26c0cb8dc66_0_0#slide=id.g26c0cb8dc66_0_0  